In [57]:
import csv 
!pip install hazm
import hazm
from hazm import Normalizer, sent_tokenize, word_tokenize 
import string
import random
import torch
from torch.utils.data import Dataset 
import gensim
import logging
import os
from gensim.models.word2vec import Word2Vec
import gensim.downloader as api
from time import time 
!pip install transformers
from transformers import AutoConfig, AutoTokenizer, AutoModel, AdamW
import numpy as np
import torch.utils.data as data_utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

In [17]:
path2 = 'persian-sentences-dataset-1.csv' 

In [18]:
path = 'Persian-WikiText-1.txt' 

### Sentence Tokenizing

In [19]:
def sentence_tokenizer(text):
    text = text.split('.')
    sentences = [] 
    for sentence in text: 
        if '!' in sentence:
            sentence = sentence.split('!')
            for subsen in sentence:
                if '؟' in subsen:
                    subsen2 = subsen.split('؟')
                    sentences.extend(subsen2)
                else:
                    sentences.append(subsen)
        elif '؟' in sentence:
            sentence_ = sentence.split('؟')
            sentences.extend(sentence_)
        else:
            sentences.append(sentence)
    return sentences
text = '.سلام! خوبی؟ می دونم سرت شلوغه. باشه! ولی بیا این طرفا. باشه ممنونم. تو خوبی؟ آره شکر، خوبم'
sentences2 = sentence_tokenizer(text)
print(sentences2) 

['', 'سلام', ' خوبی', ' می دونم سرت شلوغه', ' باشه', ' ولی بیا این طرفا', ' باشه ممنونم', ' تو خوبی', ' آره شکر، خوبم']


In [29]:
with open(path, 'r', encoding='utf-8', errors='ignore') as f:
      text = ' '.join([line.strip() for line in f.readlines() if not (('*' in line) or ('عنوان مقاله' in line))])
        
      # preprocess data
      normalizer = Normalizer()
      text = normalizer.normalize(text)
      sentences = sentence_tokenizer(text) 
                            
print(sentences[0])     
        

ویکی پدیا (کوته نوشت به صورت «وپ» و «WP») یک دانشنامه برخط چندزبانه مبتنی بر وب با محتوای آزاد و همکاری باز است که با همکاری افراد داوطلب نوشته می‌شود و هر کسی که به اینترنت و وب دسترسی داشته باشد می‌تواند مقالات آن را ببیند و ویرایش کند


### Word Embeding

In [32]:
config = AutoConfig.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
model = AutoModel.from_pretrained("HooshvareLab/bert-base-parsbert-uncased") 

Some weights of the model checkpoint at HooshvareLab/bert-base-parsbert-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [34]:
def remove_punctuation(txt):
    s = ""
    punc = string.punctuation+'،؛«»'
    for l in txt:
        if l not in punc:
            s+=l
    return s

def my_word_tokenizer(txt):
    txt = remove_punctuation(txt)
    doc = word_tokenize(txt)
    return doc

txt = 'سلام، ما که رفتیم؛ ولی شما بمونید.'
words = my_word_tokenizer(txt)
print(words) 

['سلام', 'ما', 'که', 'رفتیم', 'ولی', 'شما', 'بمونید']


In [35]:
marked_text = 'سلام دوست خوبم ما داریم میریم مدرسه'
tokenized_text = tokenizer.tokenize(marked_text)

indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

# Display the words with their indeces.
for tup in zip(tokenized_text, indexed_tokens):
    print('{:<12} {:>6,}'.format(tup[0], tup[1]))


سلام          3,132
دوست          3,584
خوبم         35,065
ما            2,179
داریم         3,194
میریم        49,824
مدرسه         5,000


### Sentences Shuffling

In [36]:
def prepare_shuffled_sentences(sentences):
    shuffled_sentences = []
    for sentence in sentences:
        sentence_words = my_word_tokenizer(sentence)
        random.shuffle(sentence_words)
        shuffled = ' '.join(sentence_words)
        shuffled_sentences.append(shuffled)
    return shuffled_sentences

sentences_2 = ['نه من نمیام.', '!بیا بریم کافه']
shuffled_sentences_2 = prepare_shuffled_sentences(sentences_2)
print(shuffled_sentences_2) 

['نمیام نه من', 'بریم بیا کافه']


In [37]:
shuffled_sentences = prepare_shuffled_sentences(sentences)
print(shuffled_sentences[0]) 

می‌تواند مبتنی با نوشت است و آن برخط و دسترسی کوته آزاد ویرایش داوطلب داشته_باشد نوشته_می‌شود اینترنت صورت ببیند ویکی WP بر همکاری به محتوای با مقالات دانشنامه باز افراد و وب به کند وپ وب چندزبانه هر را که پدیا یک و و کسی همکاری که


In [76]:
def get_bert_input(text, tokenizer, max_len=512):
   
    cls_token = '[CLS]'
    sep_token = '[SEP]'
    word_piece_list = tokenizer.tokenize(text) 
    if len(word_piece_list) > max_len-2: # notice 
        word_piece_list = word_piece_list[:510]
    word_piece_list.insert(0, cls_token)
    word_piece_list.append(sep_token)
    
    input_ids = my_word_tokenizer(word_piece_list) 
      
    input_mask  = []
    for i in range(len(input_ids)):
        input_mask.append(1) 
    while len(input_mask ) < max_len:
        input_mask.append(0)
    while len(input_ids) < max_len:
        input_ids.append(0)
    segment_ids = [0] * max_len
    return input_ids, input_mask, segment_ids

In [38]:
target_vectors = []
input_vectors = []

for sentence in sentences:
    sentence_words = my_word_tokenizer(sentence)
    indexed_tokens = tokenizer.convert_tokens_to_ids(sentence_words)
    if len(indexed_tokens) > 60:
        continue
    target_vectors.append(indexed_tokens)

print(target_vectors[0])

for sentence in sentences:
    sentence_words = my_word_tokenizer(sentence)
    indexed_tokens = tokenizer.convert_tokens_to_ids(sentence_words)
    if len(indexed_tokens) > 60:
        continue
    random.shuffle(indexed_tokens)
    input_vectors.append(indexed_tokens)


print(input_vectors[0])
print(target_vectors[0])

inputs = input_vectors
targets = target_vectors

[13827, 39512, 31487, 3695, 2031, 2353, 36562, 331, 1, 2076, 19725, 18349, 1, 6184, 2043, 4068, 2037, 6693, 1, 331, 2886, 2266, 2045, 2046, 2037, 2886, 2698, 7409, 1, 331, 2202, 3256, 2046, 2031, 3975, 331, 4068, 4405, 1, 1, 9792, 1, 2049, 9737, 331, 10077, 2299]
[9792, 1, 331, 13827, 2037, 2046, 4068, 1, 331, 2049, 19725, 2031, 2886, 2043, 4405, 3975, 4068, 331, 331, 1, 2037, 2031, 2045, 7409, 1, 2299, 39512, 1, 6693, 2886, 2698, 10077, 331, 2076, 3695, 2353, 9737, 31487, 3256, 2202, 6184, 2266, 1, 18349, 36562, 1, 2046]
[13827, 39512, 31487, 3695, 2031, 2353, 36562, 331, 1, 2076, 19725, 18349, 1, 6184, 2043, 4068, 2037, 6693, 1, 331, 2886, 2266, 2045, 2046, 2037, 2886, 2698, 7409, 1, 331, 2202, 3256, 2046, 2031, 3975, 331, 4068, 4405, 1, 1, 9792, 1, 2049, 9737, 331, 10077, 2299]


## Preparing Dataset

In [39]:
# show the size of data
print(len(inputs))

443174


### split data into train and test parts

In [40]:
train_X_data = inputs[90001:]
train_Y_data = targets[90001:]
test_X_data = inputs[:90000]
test_Y_data = targets[:90000]

print("size of train data is:", len(train_X_data))
print("size of test data is:", len(test_X_data)) 

size of train data is: 353173
size of test data is: 90000


### Convert word embedings to Tensor

In [41]:
def zero_padding(l, width):
    l.extend([0] * (width - len(l)))
    return l

In [42]:
def FindMaxLength(lst):
    list_len = []
    len_ = 0
    for item in lst:
        for x in item:
            len_ += 1
        list_len.append(len_)
        len_ = 0
    maxLength = max(list_len)
    return maxLength


my_lst = [[1, 3, 6, 7, 8, 90], [1, 2], [0], [15, 8, 9]]
max1 = FindMaxLength(my_lst)
print(max1)

6


In [43]:
max_train = FindMaxLength(train_X_data)
max_test  = FindMaxLength(test_X_data)

print(test_X_data[0])
print(test_Y_data[0])
print('max length of train sentence is:', max_train)
print('max length of test sentence is:', max_test)

[9792, 1, 331, 13827, 2037, 2046, 4068, 1, 331, 2049, 19725, 2031, 2886, 2043, 4405, 3975, 4068, 331, 331, 1, 2037, 2031, 2045, 7409, 1, 2299, 39512, 1, 6693, 2886, 2698, 10077, 331, 2076, 3695, 2353, 9737, 31487, 3256, 2202, 6184, 2266, 1, 18349, 36562, 1, 2046]
[13827, 39512, 31487, 3695, 2031, 2353, 36562, 331, 1, 2076, 19725, 18349, 1, 6184, 2043, 4068, 2037, 6693, 1, 331, 2886, 2266, 2045, 2046, 2037, 2886, 2698, 7409, 1, 331, 2202, 3256, 2046, 2031, 3975, 331, 4068, 4405, 1, 1, 9792, 1, 2049, 9737, 331, 10077, 2299]
max length of train sentence is: 60
max length of test sentence is: 60


- Add zero padding to be compatible with Tensors.

In [44]:
train_X_with_zero_padding = []
train_Y_with_zero_padding = []
for i in range(len(train_X_data)):
    train_X_with_zero_padding.append(zero_padding(train_X_data[i], 60))
    train_Y_with_zero_padding.append(zero_padding(train_Y_data[i], 60))

train_X_data = train_X_with_zero_padding
train_Y_data = train_Y_with_zero_padding

print(train_X_data[0])
print(train_Y_data[0])

[3130, 2028, 16889, 2042, 2297, 2083, 6036, 2116, 2036, 9170, 331, 10361, 19987, 2813, 7042, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[2042, 2116, 2297, 2036, 19987, 3130, 7042, 9170, 2083, 331, 2028, 16889, 2813, 10361, 6036, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [45]:
print(FindMaxLength(train_Y_data))

60


In [46]:
test_X_with_zero_padding = []
test_Y_with_zero_padding = []
for i in range(len(test_X_data)):
    test_X_with_zero_padding.append(zero_padding(test_X_data[i], 60))
    test_Y_with_zero_padding.append(zero_padding(test_Y_data[i], 60))

test_X_data = test_X_with_zero_padding
test_Y_data = test_Y_with_zero_padding

print(test_X_data[0])
print(test_Y_data[0])

[9792, 1, 331, 13827, 2037, 2046, 4068, 1, 331, 2049, 19725, 2031, 2886, 2043, 4405, 3975, 4068, 331, 331, 1, 2037, 2031, 2045, 7409, 1, 2299, 39512, 1, 6693, 2886, 2698, 10077, 331, 2076, 3695, 2353, 9737, 31487, 3256, 2202, 6184, 2266, 1, 18349, 36562, 1, 2046, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[13827, 39512, 31487, 3695, 2031, 2353, 36562, 331, 1, 2076, 19725, 18349, 1, 6184, 2043, 4068, 2037, 6693, 1, 331, 2886, 2266, 2045, 2046, 2037, 2886, 2698, 7409, 1, 331, 2202, 3256, 2046, 2031, 3975, 331, 4068, 4405, 1, 1, 9792, 1, 2049, 9737, 331, 10077, 2299, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [47]:
train_X = torch.cuda.FloatTensor([train_X_data])
train_Y = torch.cuda.FloatTensor([train_Y_data])
test_X = torch.cuda.FloatTensor([test_X_data])
test_Y = torch.cuda.FloatTensor([test_Y_data]) 

### Passing to DataLoader

In [48]:
train = data_utils.TensorDataset(train_X, train_Y)
train_loader = data_utils.DataLoader(train, batch_size=32, shuffle=True)

test = data_utils.TensorDataset(test_X, test_Y)
test_loader = data_utils.DataLoader(test, batch_size=32, shuffle=True)

### Set Hyper Parameters

In [49]:
epochs = 70
batch_size = 32
learning_rate = 1e-5

### Define OPtimizer and Loss function

In [50]:
optimizer = AdamW(model.parameters(), lr = learning_rate, eps = 1e-8)
criterion = torch.nn.CrossEntropyLoss()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
for name, layer in model.named_modules():
       print(name, layer)

### Not Trainig all layers again

In [55]:
for param in model.parameters(): 
    param.requires_grad = False 


### But train the last layer

In [56]:
for param in model.encoder.layer[11].parameters():
    param.requires_grad = True 

## Training Model

In [68]:
indexes = []
ctr = 0
for element in train_X:
    indexes.append(ctr)
    ctr += 1

In [77]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
model.train() 

loss_values = [] 
running_loss = 0.0
for epoch in range(epochs): 
    for i, (inp, out) in enumerate(train_loader):  
        # Move tensors to the configured device
        out = out.to(device)
        # Forward pass
        outputs = model(inputs_embeds=inp, output_hidden_states=False, return_dict=True)

        loss = criterion(outputs, out)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        loss_values.append(loss.item())
        
    print('Epoch [{}/{}]: Loss: {:.4f}'.format(epoch+1, epochs, loss.item()))

### Save DataLoaders

In [34]:
torch.save(train_loader, 'train_loader.pth')
torch.save(test_loader, 'test_loader.pth')